In [64]:
# Section 2: Import libraries and setup environment
import ollama
import pandas as pd
import requests
import os
import json
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [ ]:
# Load environment variables
load_dotenv()

# Get API key
TMDB_API_KEY = os.getenv("TMDB_API_KEY")
print(f"API Key found: {'Yes' if TMDB_API_KEY else 'No'}")
TMDB_BASE_URL = "https://api.themoviedb.org/3"

# Ollama API endpoint
OLLAMA_API_URL = "http://localhost:11434/api/generate"  # Default local Ollama URL

# Test the API with a simple request
if TMDB_API_KEY:
    url = f"https://api.themoviedb.org/3/movie/550?api_key={TMDB_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        print("API Key works! Successfully retrieved data for movie 'Fight Club'")
        print(f"Movie title: {response.json()['title']}")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

In [66]:
# Helper functions

def search_movies_by_title(title, page=1):
    """
    Searches TMDB for movies matching the given title.
    Returns a tuple: (list of movie dictionaries, total number of pages)
    """
    url = f"{TMDB_BASE_URL}/search/movie?api_key={TMDB_API_KEY}&language=en-US&query={title}&page={page}&include_adult=false"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data.get('results', []), data.get('total_pages', 0)
        else:
            print(f"Error searching movies: HTTP {response.status_code}")
            return [], 0
    except Exception as e:
        print(f"Exception while searching movies: {e}")
        return [], 0


In [67]:
def get_movie_details(movie_id):
    """
    Retrieves detailed information about a movie from TMDB using its ID.
    Returns a dictionary with keys such as title, director, cast, release year, and overview.
    """
    url = f"{TMDB_BASE_URL}/movie/{movie_id}?api_key={TMDB_API_KEY}&append_to_response=credits"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            movie_data = response.json()
            # Extract director(s)
            directors = [crew_member['name'] for crew_member in movie_data.get('credits', {}).get('crew', [])
                         if crew_member.get('job') == 'Director']
            # Extract top cast members (limit to 5)
            cast = [cast_member['name'] for cast_member in movie_data.get('credits', {}).get('cast', [])][:5]
            # Extract release year
            release_year = (movie_data.get('release_date', '') or 'Unknown').split('-')[0]
            return {
                'id': movie_id,
                'title': movie_data.get('title', 'Unknown'),
                'director': directors,
                'cast': cast,
                'year': release_year,
                'overview': movie_data.get('overview', '')
            }
        else:
            print(f"Error fetching movie details: HTTP {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception while getting movie details: {e}")
        return None

In [68]:
# -------------------------
# LLM-Based Pipeline Functions
# -------------------------

def get_title_candidates(user_input, model_name="deepseek-r1:7b"):
    """
    Uses the LLM to return a list of candidate corrected movie titles based on the user input.
    The LLM should return JSON like:
    [
      {"title": "Corrected Title 1"},
      {"title": "Corrected Title 2"},
      {"title": "Corrected Title 3"}
    ]
    """
    prompt = f"""
    I have a user who typed a movie title that might be incorrect or misspelled: "{user_input}".
    Please provide a JSON list of up to 3 possible corrected movie titles that the user might have intended.
    Each item should be an object with a "title" key.
    Return only valid JSON. For example:
    [
      {{"title": "Correct Title 1"}},
      {{"title": "Correct Title 2"}},
      {{"title": "Correct Title 3"}}
    ]
    """
    try:
        response = ollama.chat(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        response_text = response.get("message", {}).get("content", "")
        json_match = re.search(r'(\[.*\])', response_text, re.DOTALL)
        if json_match:
            candidates = json.loads(json_match.group(1))
            return [item["title"] for item in candidates if "title" in item]
        else:
            print("No valid JSON found in LLM response for title candidates.")
            return []
    except Exception as e:
        print(f"Error in LLM call for title candidates: {e}")
        return []

In [ ]:
# Test the get_title_candidates function
user_input = "Fite Clib"  # An intentionally misspelled title
candidates = get_title_candidates(user_input)
print("LLM Title Candidates for '", user_input, "':", candidates)


In [70]:
def find_movie_using_llm(user_input, model_name="deepseek-r1:7b"):
    """
    Uses an LLM-driven pipeline to correct a potentially incorrect movie title.
    1. The LLM suggests candidate corrected titles.
    2. If only one candidate is returned, it queries TMDB for that candidate and returns the best match.
    3. If multiple candidates are returned, it searches TMDB for each candidate,
       passes the combined candidate list to the LLM to choose the best match,
       and then retrieves detailed movie info using the selected TMDB ID.
    """
    # Step 1: Get candidate corrected titles from the LLM.
    candidates = get_title_candidates(user_input, model_name)
    if not candidates:
        print("LLM did not provide candidate titles; falling back to user input.")
        candidates = [user_input]
    else:
        print("LLM candidate titles:", candidates)
    
    # Step 2: Search TMDB for each candidate title.
    tmdb_results = []
    for title in candidates:
        movies, _ = search_movies_by_title(title)
        tmdb_results.extend(movies)
    
    if not tmdb_results:
        print("No movies found in TMDB for the candidate titles.")
        return None
    
    # Reduce the candidate list to the top 5 by popularity.
    tmdb_results = sorted(tmdb_results, key=lambda x: x.get("popularity", 0), reverse=True)[:5]
    
    # Prepare candidate info for the LLM.
    candidate_info = []
    for movie in tmdb_results:
        candidate_info.append({
            "id": movie.get("id"),
            "title": movie.get("title"),
            "release_date": movie.get("release_date"),
            "popularity": movie.get("popularity"),
            "overview": movie.get("overview")
        })
    candidates_json = json.dumps(candidate_info, indent=2)
    
    # Step 3: Ask the LLM to choose the best matching movie from the candidates.
    prompt = f"""
    I have searched the TMDB database for candidate movies based on the user input "{user_input}" and the following corrected candidate titles: {candidates}.
    The candidate movies are provided below in JSON format:
    
    {candidates_json}
    
    Please review the candidates and determine the best matching movie title that the user intended.
    Return your answer in the following JSON format only (do not include any extra explanation):
    
    {{
      "best_movie_id": <the TMDB id of the best matching movie>
    }}
    """
    
    try:
        response = ollama.chat(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
    except Exception as e:
        print("Error calling the LLM to choose the best candidate:", e)
        return None
    
    response_text = response.get("message", {}).get("content", "")
    json_match = re.search(r'({[\s\S]*})', response_text)
    if json_match:
        try:
            llm_result = json.loads(json_match.group(1))
        except json.JSONDecodeError:
            print("Error decoding JSON from the LLM response.")
            return None
    else:
        print("No valid JSON found in the LLM response for best candidate.")
        return None
    
    best_movie_id = llm_result.get("best_movie_id")
    if not best_movie_id:
        print("LLM did not return a valid movie ID.")
        return None
    
    # Step 4: Retrieve detailed information for the selected movie using TMDB.
    best_movie_details = get_movie_details(best_movie_id)
    return best_movie_details


In [ ]:
# Test the complete pipeline using the LLM
user_input = "Fite Clib"  # Example misspelled input
best_movie = find_movie_using_llm(user_input)
if best_movie:
    print("Final Movie Details as determined by the LLM Pipeline:")
    print(json.dumps(best_movie, indent=2))
else:
    print("Could not determine a correct movie from the input.")

In [ ]:
# More test cases
test_inputs = [       
    "Avtar",             # Avatar (misspelled)
    "Titannic",          # Titanic (misspelled)
    "Incepton",          # Inception (misspelled)
    "Str Wars",          # Star Wars (misspelled)
    "Lord of the Rngs",  # Lord of the Rings (misspelled)
    "Bladrunner",        # Blade Runner (misspelled)
    "Matirx",            # The Matrix (misspelled)
    "Avatar",            # Correctly spelled for control check
]

# Iterate through the test inputs and run the complete LLM pipeline.
for input_title in test_inputs:
    print(f"\nTesting input: {input_title}")
    result = find_movie_using_llm(input_title)
    if result:
        print(json.dumps(result, indent=2))
    else:
        print("No result returned for this input.")
